# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに関連しており、スキル評価システムの公正性を評価するために作成されたものです。Notebookでは、競技の評価システムの一部としてTrueSkillを使用しており、特にボットのスキルをシミュレートし、ランキングシステムがパフォーマンスをどのように評価するかを実験しています。

### 取り組んでいる問題
このNotebookは、Kaggleのコンペティションのランキングシステムが公平であるか疑問を持ち、それを評価するためにデータを操作して分析しています。特に、TrueSkillを用いたスキル評価が実際の競技でどのように機能するかを調査しています。また、新しいボットの提出や、競技内での勝敗によってスキル評価が変化するという要素に注目しています。

### 使用している手法やライブラリ
1. **TrueSkillライブラリ**: このNotebookの主な方法論としてTrueSkillを使用し、ボットのスキルやレーティングを管理します。これを活用して、勝敗に基づいてレーティングを更新します。
   
2. **データ操作ライブラリ**: `pandas`を使って、ゲームデータの読み込み、操作、分析を行っています。

3. **シミュレーション**: Notebook内でシミュレーションを行い、ボットのスキル分布や引き分けの確率を分析します。また、シミュレーションされたデータに基づいてプレイヤーのパフォーマンスを視覚化しています。

4. **マルチプロット作成**: `matplotlib`を用いて、プレイヤーのスキルやTrueSkillのμ値の変化を視覚化し、シミュレーションの結果と実際のリーダーボードデータを比較しています。

5. **ゲームプレイのシミュレーション**: チームマッチをシミュレートするための関数を定義しており、異なるスキル分布やシードを用いて多数のゲームを実行し、その結果を考察しています。

### 結論
Notebookは、Kaggleの「LLM 20 Questions」コンペティションにおける評価システムとその公平性を理解するための重要な実験を行っています。試行錯誤を通じて得た知見は、ボット間の評価や実際の競技におけるスキルの表現に影響を与え、コンペティションのランキングシステムがパフォーマンスよりも偶然性に大きく依存していることを示唆しています。

---


# 用語概説 
以下は、ノートブックの内容に基づき、機械学習・深層学習の初心者がつまずきそうな専門用語とその簡単な解説です。

1. **ガウス分布 (Gaussian distribution)**:
   中心が平均値で左右対称のベルカーブの形を持つ確率分布。データが平均値の周りに集中し、標準偏差によって散らばり具合が決まる統計的性質を持つ。

2. **レーティング (Rating)**:
   競技やゲームの中でプレイヤーのスキルを数値で評価したもので、特定のスキル評価システム（例: TrueSkill, Glicko）に基づいている。これによりプレイヤー間の比較が可能になる。

3. **不確実性 (Uncertainty)**:
   特定の推定がどれだけの誤差を含むかを示す指標。例えば、スキル評価の不確実性は、プレイヤーの実力が非常に変動する可能性を示す。

4. **エピソード (Episode)**:
   競技やシミュレーション内での一連の活动のこと。通常、ゲームまたはラウンドの単位で進行し、特定の条件（例えば、勝利、敗北）を評価する。

5. **σ (sigma)**:
   正規分布の標準偏差を示し、データの広がりを表す。特にTrueSkillの文脈では、レーティングの不確実性を表すパラメータとして機能。

6. **引き分け確率 (Draw probability)**:
   ゲームにおいて両チームまたはプレイヤーが同じ結果で終わる確率。特にTrueSkillでは、評価戦略に影響を与える重要な要素。

7. **β (beta)**:
   TrueSkillのパラメータの一つで、勝利確率を保証する距離を示す。競技の中でプレイヤーのレーティング更新に使用される。

8. **τ (tau)**:
   レーティングの変化を制限する動的要因を示すTrueSkillのパラメータ。これにより、評価の安定性が保たれる。

9. **ダミーボット (Dummy bot)**:
   知識や戦略がほとんどないか、全く持っていないプレイヤーのこと。リアルな競争環境の中で、他のボットのテストやスコアリング時に使われることが多い。

10. **TrueSkill**:
    Microsoftが開発したレーティングシステムで、プレイヤーのスキルを推定するためにガウス分布を基にしている。特にチーム戦において効果的で、プレイヤー間の相対的スキルを比較するのに役立つ。

これらの用語は、ノートブック内のコンセプトを理解するために重要であり、初心者が実務経験なしに出遭うことの少ない特有の用語です。

---


[LLM 20 Questions コンペティション](https://www.kaggle.com/competitions/llm-20-questions/overview) のランキングシステムの公平性は、いくつかの議論で何度も疑問視されているため、それを評価するために小さな遊びのノートブックを書くことにしました。数学を評価するのではなく、データを操作して遊んでみましょう。

# ランキングシステムの公式説明

執筆時点（7月17日）で、コンペティションの概要には次のように示されています：
> ### 評価
>
> 毎日、あなたのチームは最大5つのエージェント（ボット）をコンペティションに提出することができます。各提出物は、リーダーボード上の他のボットと対戦を行いますが、これらのボットは同様のスキル評価を持っています。時間が経つにつれて、勝利すればスキル評価が上がり、敗北すれば評価が下がり、引き分けの場合はお互いの評価が均一化されます。
>
> このコンペティションは、協力的な2対2の形式で行われます。あなたのボットは、同程度のスキルを持つ他のボットとランダムにペアになり、別のランダムなペアと対戦します。各ペアでは、一方のボットが質問者、もう一方が回答者にランダムに割り当てられます。ペアとして勝敗が決まるため、協力し合うことが奨励されます！
>
> 提出されたすべてのボットはコンペティション終了時までプレイし続け、新しいボットはより頻繁に選択されるようになっています。3つのアクティブな提出物があると、古いものが非アクティブ化されます。リーダーボード上には最高のスコアを持つボットだけが表示されますが、すべての提出物の進捗はあなたの提出ページで確認可能です。
>
> 各提出物には、ガウス分布N(μ,σ2)でモデル化された推定スキル評価があり、μは推定スキル、σはその推定の不確実性を表し、時間の経過とともに減少します。
>
> 提出物をアップロードすると、まずその提出物が自身のコピーと対戦する検証エピソードをプレイし、正常に動作することを確認します。エピソードが失敗した場合、その提出物はエラーとしてマークされ、原因を特定するためにエージェントのログをダウンロードできます。そうでない場合は、μ0=600で初期化され、継続的な評価のためのプールに参加します。また、この時、アクティブなエージェントの総数が3つを超える場合は古いエージェントが非アクティブ化されます。
>
> ### ランキングシステム
> 
> エピソードが終了すると、そのエピソードに参加したすべてのボットの評価推定値が更新されます。一方のボットペアが勝利した場合、そのペアのμは増加し、対戦相手のμは減少します。結果が引き分けの場合は、μ値はそれぞれの平均値に近づきます。更新の大きさは、以前のμ値に基づく予想結果からの偏差および各ボットの不確実性σに比例します。また、結果から得られた情報量に基づいてσ項も減少します。エピソードでの勝利または敗北によって得られるスコアは、スキル評価の更新には影響しません。
> 
> ### 最終評価
> 
> 2024年8月13日の提出締め切り時点で、提出物はロックされます。2024年8月13日から8月27日まで、未公開の新しい単語セットに対してエピソードを実行し続けます。この期間中、あなたの3つのアクティブな提出物のみがリーダーボードの対象となります。この期間の終了時に、リーダーボードは確定します。

# TrueSkill

私は評価システムについてあまり詳しくありません。簡単な検索によると、$\mu$ **および** $\sigma$ パラメータを持つシステムの候補として [Glicko評価システム](https://en.wikipedia.org/wiki/Glicko_rating_system) が考えられます。この結論に至った経緯は覚えていませんが（チームプレイに関係していたのかもしれません）、より可能性のある候補は [TrueSkill](https://en.wikipedia.org/wiki/TrueSkill) です。詳細には触れませんので、興味のある方はウィキペディアのリンクを参照してください。ビデオゲームの世界では非常に人気があるようです。 [TrueSkill 2](https://www.microsoft.com/en-us/research/uploads/prod/2018/03/trueskill2.pdf) もありますが、第一印象ではその改善が私たちのコンペティションには関連しているようには見えません。

## インストール

幸いなことに、[Pypi](https://pypi.org/project/trueskill/) に [TrueSkillのPythonライブラリ](https://github.com/sublee/trueskill) があるので、インストールしてみましょう。

In [ ]:
!pip install trueskill  # TrueSkillライブラリをインストールします。これは、ボットのスキル評価を行うために使います。

プレイヤーのレーティングは `Rating` クラスで表され、ゲームは `rate()` メソッドを使って実行されます。

In [ ]:
from trueskill import Rating, rate  # TrueSkillからRatingクラスとrate関数をインポートします。
# Ratingクラスはプレイヤーのスキルを表すために使用され、rate関数はゲームの結果に基づいてプレイヤーのレーティングを更新するために使われます。

## ゲームを始めましょう

4人のプレイヤーを $\mu=600$ で定義しましょう：

In [ ]:
player1 = Rating(mu=600)  # プレイヤー1のレーティングを初期化します（μ=600）。
player2 = Rating(mu=600)  # プレイヤー2のレーティングを初期化します（μ=600）。
player3 = Rating(mu=600)  # プレイヤー3のレーティングを初期化します（μ=600）。
player4 = Rating(mu=600)  # プレイヤー4のレーティングを初期化します（μ=600）。

teamA = [ player1, player2 ]  # チームAをプレイヤー1とプレイヤー2で構成します。
teamB = [ player3, player4 ]  # チームBをプレイヤー3とプレイヤー4で構成します。

さて、彼らにゲームをプレイさせましょう：

In [ ]:
rate(
    [ teamA, teamB ],  # チームAとチームBのレーティングを更新します。
    [ 0, 1 ]  # teamAはteamBよりもランクが低いため、teamAが勝利したことを示します。
)

`rate()` 関数は、各プレイヤーの新しいレーティングをチームごとに返します。

## シンプルな表示関数

ゲームの結果を読みやすくするための関数を定義すると便利です：

In [ ]:
from typing import Tuple  # Tuple型をインポートします。これは、プレイヤーのペアを表すために使います。

def play_game(
    teamA: Tuple[Rating, Rating],  # チームAのプレイヤーのタプル。
    teamB: Tuple[Rating, Rating],  # チームBのプレイヤーのタプル。
    outcome: Tuple[int, int]  # ゲームの結果を示すタプル（0: 1位、1: 2位）。
):
    """
    2つのチーム間でゲームをプレイし、プレイヤーのレーティングへの影響を表示します。
    
    パラメータ
    ---------
        teamA: Tuple[Rating, Rating]
            2人のプレイヤーからなるチーム。
        teamB: Tuple[Rating, Rating]
            2人のプレイヤーからなるチーム。
        outcome: Tuple[int, int]
            ゲーム内のプレイヤーのランク付け（0: 最初、1: 次）。
    """
    new_ratings = rate(
        [ teamA, teamB ],  # チームAとチームBのレーティングを更新します。
        outcome  # ゲームの結果を引数として渡します。
    )
    for old,new in zip(teamA, new_ratings[0]):  # チームAの古いレーティングと新しいレーティングを取得します。
        delta = int(new.mu - old.mu)  # レーティングの変化量を計算します。
        print(f"{old.mu:.0f} -> {new.mu:.0f} ({'+' if delta >=0 else ''}{delta})")  # レーティングの変化を表示します。
    print("vs")  # 対戦していることを示します。
    for old,new in zip(teamB, new_ratings[1]):  # チームBの古いレーティングと新しいレーティングを取得します。
        delta = int(new.mu - old.mu)  # レーティングの変化量を計算します。
        print(f"{old.mu:.0f} -> {new.mu:.0f} ({'+' if delta >=0 else ''}{delta})")  # レーティングの変化を表示します。

前回のプレイヤーに対してこれを適用してみましょう：

In [ ]:
play_game(teamA, teamB, [0, 1])  # チームAが勝利した結果を表示します。ここでは、チームAが1位、チームBが2位という結果を示しています。

## シグマの特定

現在、競技はデフォルト値の $\sigma$ を使用していないことが明らかになってきました。TrueSkillの推奨では、$\sigma = \frac{\mu}{3}$ を設定することになっていますが、これらのパラメータで少し遊んでみた結果、この競技においてはむしろ $\sigma = \frac{\mu}{2} = 300$ に設定されていると考えています。

実際のエピソードの結果を再現してみましょう。こちらは私の最初の提出物の最初の勝利の結果です：

> [1位] gguillard 600 (+123) vs [1位] JavaZero 570 (+34) vs
>
> [3位] atom1231 577 (-72) vs [3位] KKY 464 (-44)

少しの試行錯誤を行い、私の $\sigma$ が初期値の300からそれほど遠くないと仮定すると、一致するパラメータの組み合わせを見つけるのは簡単です：

In [ ]:
play_game(
    [
        Rating(mu=600, sigma=295),  # プレイヤー1のレーティング（μ=600、σ=295）。
        Rating(mu=570, sigma=156)    # プレイヤー2のレーティング（μ=570、σ=156）。
    ],
    [
        Rating(mu=577, sigma=225),  # プレイヤー3のレーティング（μ=577、σ=225）。
        Rating(mu=464, sigma=176)    # プレイヤー4のレーティング（μ=464、σ=176）。
    ],
    [ 0, 1 ]  # 最初のチームが勝利したことを示します。
)

## 引き分けの確率

それでは、得点システムはTrueSkillで $\mu=600$ と $\sigma=300$ なのでしょうか？　実際にはそうではありません。これらの設定は引き分けの場合にうまく機能せず、変動が競技で観察されるものよりもはるかに大きくなります（通常は0から±数単位程度）：

In [ ]:
play_game(
    [
        Rating(mu=600, sigma=295),  # プレイヤー1のレーティング（μ=600、σ=295）。
        Rating(mu=570, sigma=156)    # プレイヤー2のレーティング（μ=570、σ=156）。
    ],
    [
        Rating(mu=577, sigma=225),  # プレイヤー3のレーティング（μ=577、σ=225）。
        Rating(mu=464, sigma=176)    # プレイヤー4のレーティング（μ=464、σ=176）。
    ],
    [ 1, 1 ]  # 勝者なし（引き分け）の結果を示します。
)

ルールを再度読み返すと、重要なポイントは次の通りです：

> 結果によって得られた情報量に基づいて、σ項も減少します。

最初は、引き分けの場合にσが100倍の係数で手動で減少すると思い、うまく機能しているように見えました。その後、`help(trueskill)`を読んでみると、TrueSkill内で既に考慮されている可能性があることに気付きました。なぜなら、μとσの更新を担当するV関数とW関数には、それぞれ「勝ち」**と**「引き分け」のバージョンがあるからです。

実際、いくつかの追加パラメータを持つTrueSkill環境を定義することができ、その中には`draw_probability`も含まれています。これは興味深いことです。なぜなら、私たちはこのコンペティションにおけるゲームの引き分け確率が… *かなり高い*ことを知っているからです。

これを計算しましょう。シミュレーションに必要になります。私たちは、@waechterの[LLM 20 Questions - Games dataset](https://www.kaggle.com/code/waechter/llm-20-questions-games-dataset)を使用します。

In [ ]:
import pandas as pd  # pandasライブラリをインポートします。データ操作に便利です。
from ast import literal_eval  # literal_evalをインポートします。文字列をリストや辞書に変換するために使用します。

# ゲームデータセットをCSVファイルから読み込みます。
games_df = pd.read_csv(
    "/kaggle/input/llm-20-questions-games-dataset/LLM-20Questions-games.csv",
    converters = {
        col_list: literal_eval  # 指定した列をリストや辞書に変換します。
        for col_list in ["answers", "questions", "guesses"]
    }
)

# guesser_SubmissionIdが0でない行のみを取得します。これは、当日の提出物がまだラベル付けされていないようです。
games_df = games_df[games_df["guesser_SubmissionId"]!=0]  
# CreateTime列を日付時刻形式に変換します。
games_df["CreateTime"] = pd.to_datetime(games_df["CreateTime"], format="%m/%d/%Y %H:%M:%S")  # 日付のフォーマットを指定して変換します。

このデータフレームには、各ゲームに対して2行が含まれています。各チームに1行ずつで、ブール値の「guessed」列があります。引き分けの場合、「guessed」は両チームで同じになります（ただし、両チームが同じラウンドで勝った3ゲームは無視しても問題ありませんが…）。

In [ ]:
games_df.groupby("game_num")["guessed"].sum().value_counts()  # ゲーム番号でグループ化し、各ゲームの"guessed"の合計を計算し、その値のカウントを求めます。これにより、各ゲームでの引き分けの確率を調べることができます。

In [ ]:
(144607 + 3) / (144607 + 1208 + 3)  # 引き分けの確率を計算します。これにより、勝敗の結果をもとに引き分けの確率を求めています。

したがって、引き分けの確率は $p \simeq 0.9917$ です。確かにかなり高いですね…。より正確なデータで後で更新します（ネタバレ：結果はさらに悪化するだけです）。

## ベータとタウ

ドキュメントには、他に興味深いかもしれない二つのパラメータが言及されています：

>  :param beta: 約76%の勝利確率を保証する距離。
>               推奨値は ``sigma`` の半分。
>
>  :param tau: レーティングの固定を制限する動的要因。
>              推奨値は ``sigma`` パーセント。

推奨値を使用しましょう。すなわち、$\beta = \frac{\sigma}{2} = 150$ と $\tau = \frac{\sigma}{100} = 3$ です。

さて、新しいTrueSkill環境を定義しましょう：

In [ ]:
from trueskill import setup  # TrueSkillのsetup関数をインポートします。これにより、TrueSkill環境を設定します。

# 新しいTrueSkill環境を設定します。
setup(
    mu = 600,  # 初期スキルレーティング。
    sigma = 300,  # 初期の不確実性。
    beta = 150,  # 勝利確率を保証する距離。
    tau = 3,  # レーティングの固定を制限する動的要因。
    draw_probability = 0.9917  # 引き分けの確率。
)

再度、正しい $\sigma$ を得るために少し調整が必要です：

In [ ]:
play_game(
    [
        Rating(mu=600, sigma=144),  # プレイヤー1のレーティング（μ=600、σ=144）。
        Rating(mu=570, sigma=76)    # プレイヤー2のレーティング（μ=570、σ=76）。
    ],
    [
        Rating(mu=577, sigma=109),  # プレイヤー3のレーティング（μ=577、σ=109）。
        Rating(mu=464, sigma=85)    # プレイヤー4のレーティング（μ=464、σ=85）。
    ],
    [ 0, 1 ]  # 最初のチームが勝利したことを示します。
)

しかし、引き分け後のレーティング更新は、実際にコンペティションで観察されるものと much より整合しています：

In [ ]:
play_game(
    [
        Rating(mu=600, sigma=144),  # プレイヤー1のレーティング（μ=600、σ=144）。
        Rating(mu=570, sigma=76)    # プレイヤー2のレーティング（μ=570、σ=76）。
    ],
    [
        Rating(mu=577, sigma=109),  # プレイヤー3のレーティング（μ=577、σ=109）。
        Rating(mu=464, sigma=85)    # プレイヤー4のレーティング（μ=464、σ=85）。
    ],
    [ 1, 1 ]  # 勝者なし（引き分け）の結果を示します。
)

私たちは、LLM 20 Questionsコンペティションで使用されているスコアリングシステムと非常に似たものを構築できたように思います。パラメータが*正確に*同じものである可能性は低く、いずれにせよ引き分けの確率は一定数のゲームが行われるまで推測できなかった可能性があります（おそらく継続的に更新されているのでしょうか？ – ちなみに関数を受け入れることもできます）が、これらの設定がゲームの結果を再現する能力が偶然であったとは思えません。

# コンペティションのモデリング

## 勝利確率

では、どうやって全てのプレイヤーのボードを生成し、ランダムにチームを組んでプレイさせるのでしょうか？ :D

まずは、現実的な勝利確率を抽出しましょう。再度*LLM 20 Questions - Gamesデータセット*を使用して、各提出物の`nb_guessed / nb_played`比率を計算します。

しかし、現実的なスキル分布を得るために、まずは少ないゲームプレイ数（たとえば10回）の提出物をフィルタリングする必要があります：

In [ ]:
nb_games_played = games_df.groupby("guesser_SubmissionId").size()  # 各提出物のゲームプレイ数をカウントします。
nb_games_played = nb_games_played[nb_games_played>=10]  # ゲームプレイ数が10回以上の提出物のみを残します。

In [ ]:
games_df = games_df[games_df["guesser_SubmissionId"].isin(nb_games_played.index)]  # ゲームプレイ数が10回以上の提出物にフィルタリングされたデータフレームを作成します。これにより、対象となる提出物のみが残ります。

競技は1競技者あたり最大3つの提出物のみを許可しているため、最後の3つを除いてすべてのボットを削除する必要があります。中には100以上の提出物を持っている人もいます！

In [ ]:
games_df.groupby("guesser")["guesser_SubmissionId"].nunique().sort_values(ascending=False).head()  # 各予想者についてユニークな提出物の数をカウントし、降順でソートして上位の結果を表示します。これにより、最も多くの提出物を持つ予想者を特定できます。

In [ ]:
last_bots = games_df.sort_values(
    by = "CreateTime"  # 提出物の作成時間でソートします。「guesser」でソートしないのは、いくつかの提出物が複数の名前を持っているためです。
).drop_duplicates(
    subset = "guesser_SubmissionId",
    keep = "last"  # 最後の提出物を保持します。
).groupby(
    [ "guesser" ]
)[[
    "guesser",
    "guesser_SubmissionId",
    "CreateTime"
]].tail(3).sort_values(
    by = "guesser"
)["guesser_SubmissionId"].values  # 各予想者につき最後の3つの提出物を取得し、その提出物のIDを取得します。

In [ ]:
last_games_df = games_df[games_df["guesser_SubmissionId"].isin(last_bots)]  # 最後のボットに対応するゲームデータをフィルタリングして取得します。これにより、最新の提出物に関連するデータフレームを作成します。

引き分けの確率を更新する時間です：

In [ ]:
last_games_df.groupby("game_num")["guessed"].sum().value_counts()  # ゲーム番号でグループ化し、各ゲームの"guessed"の合計を計算し、その値のカウントを求めます。これにより、引き分けの確率を更新するためのデータを得ることができます。

In [ ]:
(123874 + 1) / (123874 + 546 + 1)  # 引き分けの確率を計算します。これにより、勝利と引き分けの結果をもとに引き分けの確率を求めています。

先に述べたように、引き分けの確率は以前よりもさらに悪化し、$p \simeq 0.9956$ となっています…。

実際のスキル分布を計算するために、「良いボット」と「ダミーボット」を区別する必要があります。

In [ ]:
goodbots = set(
    last_games_df[
        last_games_df.groupby("guesser_SubmissionId")["guessed"].transform("any")  # 「guessed」がTrue（予想が成功）であるボットを識別します。
    ]["guesser_SubmissionId"].unique()  # ユニークなボットのIDを取得します。
)  # 成功した予想を行ったボットの集合を作成します。

In [ ]:
badbots = set(last_games_df["guesser_SubmissionId"].unique()) - goodbots  # ユニークなボットのIDから「良いボット」を取り除き、「ダミーボット」の集合を作成します。

In [ ]:
len(goodbots), len(badbots)  # 「良いボット」と「ダミーボット」のそれぞれの数を取得して表示します。これにより、両者の比較が可能になります。

今度は、良いボットのチームをすべて考慮しましょう。相手がダムであることは気にしません。なぜなら、信頼できるチームメイトがいればキーワードを見つけることができるからです。また、他のチームが先にキーワードを見つけてしまったために、チームが見つける時間がなかったセッションは無視します。これは私たちのシミュレーションにバイアスをかけるだけです。一方で、チームメイトがダミーボットカテゴリに属するにも関わらずキーワードが推測された場合はそのインスタンスを保持します。なぜなら、たまたまそのチームメイトにはまだ勝つチャンスがなかっただけかもしれないからです。偶然の話ですが、これらのケースを削除すると統計が大幅に減少します。

In [ ]:
fair_games = last_games_df["guesser_SubmissionId"].isin(goodbots) * (  # 良いボットのいずれかが「guesser」にいるゲームを特定します。
    last_games_df["answerer_SubmissionId"].isin(goodbots) + last_games_df["guessed"]  # 良いボットが「answerer」にいるか、推測が成功したかどうかを確認します。
)
fair_games_df = last_games_df.loc[fair_games]  # 条件を満たすゲームデータを抽出します。
fair_games_df = fair_games_df[(fair_games_df["guessed"]) | (fair_games_df["nb_round"]==20)]  # 推測が成功したか、ラウンド数が20であるものを保持します。

In [ ]:
skills_df = fair_games_df.groupby("guesser_SubmissionId").agg(
    guessed = ("guessed", "sum"),  # 各「guesser_SubmissionId」ごとに推測の合計を計算します。
    played = ("guessed", "size")   # 各「guesser_SubmissionId」ごとにプレイされたゲームの数を計算します。
).sort_index()

skills_df["skill"] = skills_df["guessed"] / skills_df["played"]  # スキルを推測数とプレイ数の比率として計算します。
skills_df.sort_values(by = "skill", ascending = False, inplace = True)  # スキルが高い順にデータフレームをソートします。
skills_df.head()  # スキルの高い上位のボットの情報を表示します。

不自然な確率を避けるために、$N = 10$ 回未満のフェアゲームを持つボットを削除しましょう（すべてのゲームに対してはすでにこれを行いましたが、フェアゲームは別の話です）：

In [ ]:
skills_df = skills_df[skills_df["played"]>=10]  # プレイ数が10回以上のボットのみを残します。
skills_df.head()  # 条件を満たすボットの情報を表示します。

最後に、最良の推測者がキーワードを見つける確率を98%のスーパープッシャーにすることで、スキルの動的範囲を広げましょう。これは非常に楽観的です！しかし、これはまた、競技の終わりに近づくにつれてより良いボットへと進化することを反映したものともなります。

In [ ]:
BEST_SKILL = .98  # 最良のスキルを98%に設定します。
skills_df["skill"] *= BEST_SKILL / skills_df["skill"].iloc[0]  # スキルの値を更新し、最良のスキルと比較して調整します。

これで、現在の（正規の）ボットのグローバルなスキル分布をある程度リアルに表現できました。これを見てみましょう：

In [ ]:
import matplotlib.pyplot as plt  # データの可視化のためにmatplotlib.pyplotをインポートします。これを使用してスキル分布をプロットします。

In [ ]:
plt.figure(figsize=(12,6))  # プロットのサイズを設定します。
_ = plt.hist(skills_df["skill"], bins=15)  # スキル分布のヒストグラムを作成します。
_ = plt.xlabel("フェアなチームメイトに対してキーワードを見つける確率")  # x軸のラベルを設定します。
_ = plt.ylabel("ボットの数")  # y軸のラベルを設定します。
_ = plt.title("良いボットのスキル分布")  # プロットのタイトルを設定します。

これにより、明確なリーダーボードが得られるはずです！

## プレイヤークラス

それでは、レーティング履歴を追跡するためのPlayerクラスを作成しましょう：

In [ ]:
from typing import Optional  # Optionalをインポートします。これは、引数がNoneである可能性を示すために使用します。
from trueskill import TrueSkill  # TrueSkillをインポートします。これはレーティングシステムの一部です。

class Player:
    def __init__(
        self,
        skill: float,  # プレイヤーのスキルを初期化します。
        env: Optional[TrueSkill] = None  # 今は無視しますが、後で役立ちます。
    ):
        self.skill = skill  # スキルを設定します。
        self.rating = Rating() if env is None else env.Rating()  # 環境のデフォルト値（600,300）で開始します。
        self.history = []  # レーティング履歴を保持するリストを初期化します。
        
    def update(self, new_rating):  # 新しいレーティングで更新するメソッドを定義します。
        self.history.append(self.rating)  # 現在のレーティングを履歴に追加します。
        self.rating = new_rating  # 新しいレーティングを設定します。

## プレイヤープール

次に、リアルなスキルの範囲を持つプレイヤーのプールを構築し、ダミーボットの数も加えます。良いボットのスキル評価に考慮される数が大幅に減少したため、ダミーボットの数にも同じ減少比率を適用するのが公平に思えますが、シミュレーションの結果にはあまり影響しないはずです（実際、悪化するかもしれません…）。興味があれば、遊んでみてください。

In [ ]:
reduction_ratio = len(skills_df) / len(goodbots)  # 良いボットに対するスキル評価の減少比率を計算します。

players = { ii: Player(s) for ii,s in enumerate(skills_df["skill"].values) }  # スキルに基づいてプレイヤーのプールを作成します。
players.update({ len(players)+ii: Player(0) for ii in range(int(len(badbots)*reduction_ratio)) })  # ダミーボットを追加します。

len(players)  # プレイヤーの総数を表示します。

実際のリーダーボードにいるプレイヤーよりもまだ多くのプレイヤーがいますが、これはボットの数であることを忘れないでください。リーダーボードの各競技者は最大3つのアクティブな提出物を持つことができます。

## ゲーム関数

最後に、勝利確率を考慮したゲーム関数を定義する必要があります。

In [ ]:
import random  # ランダムな数値を生成するためのrandomライブラリをインポートします。これを使用してゲーム内の動的な要素を作成します。

In [ ]:
def play_match(
    teamA: Tuple[Player, Player],  # チームAのプレイヤーを定義します。
    teamB: Tuple[Player, Player],  # チームBのプレイヤーを定義します。
    env: Optional[TrueSkill] = None,  # 環境をオプションで設定します。
    debug: bool = False  # デバッグ用のフラグを設定します。
):
    """
    2つのチーム間でマッチをプレイします。
    
    パラメータ
    ---------
        teamA: Tuple[Player, Player]
            2人のプレイヤーからなるチームA。
        teamB: Tuple[Player, Player]
            2人のプレイヤーからなるチームB。
        debug: boolean
            デバッグ用のフラグ。
    """
    if debug:
        print("プレイヤーのスキル :", [p.skill for p in teamA+teamB])  # プレイヤーのスキルを表示します。
        print("プレイヤーのレーティング :", [tuple(p.rating for p in teamA), tuple(p.rating for p in teamB)])  # プレイヤーのレーティングを表示します。
    
    # 各チームのプレイヤーがダミーボットであれば、そのチームはキーワードを見つけられない
    # さもなくば、チームは推測者のスキルがランダムな数値よりも高ければキーワードを見つける
    # （コード内の逆のロジックに注意）
    ranks = [
        1 if teamA[0].skill * teamA[1].skill == 0 or random.random() > teamA[0].skill else 0,
        1 if teamB[0].skill * teamB[1].skill == 0 or random.random() > teamB[0].skill else 0
    ]
    
    # 新しいレーティングはゲームの結果です
    rating_function = rate if env is None else env.rate  # 環境に応じたレーティング関数を設定します。
    new_ratings = rating_function(
        [
            [ p.rating for p in teamA ],  # チームAのプレイヤーのレーティングを取得します。
            [ p.rating for p in teamB ]   # チームBのプレイヤーのレーティングを取得します。
        ],
        ranks  # 勝率を引数にします。
    )
    
    # 全てのプレイヤーのレーティングを更新します
    for p,r in zip(teamA, new_ratings[0]):
        p.update(r)  # チームAの各プレイヤーのレーティングを更新します。
    for p,r in zip(teamB, new_ratings[1]):
        p.update(r)  # チームBの各プレイヤーのレーティングを更新します。
        
    if debug:
        print("ゲーム結果 :", ranks)  # ゲームの結果を表示します。
        print("新しいレーティング :", new_ratings)  # 新しいレーティングを表示します。
    
    return  # 終了します。

そして、出来上がりです！  競技をシミュレーションするために必要なものはすべて揃いました。こちらが最初のゲームです：

In [ ]:
play_match(
    [players[0], players[1]],  # プレイヤー0とプレイヤー1のチームA
    [players[2], players[3]],  # プレイヤー2とプレイヤー3のチームB
    debug = True  # デバッグ情報を表示します。
)

# 自分自身の競技シミュレーションを作成

便利さと異なる設定を簡単にテストするために、プレイヤーの数やスキル、ゲームの数、TrueSkillの設定をパラメータ化できる競技クラスを定義し、レーティングの履歴を含むデータフレームを提供します。

In [ ]:
from typing import List  # List型をインポートします。リストの型注釈に使用します。
from tqdm import tqdm  # tqdmをインポートします。進行状況バーを簡単に表示できます。

In [ ]:
class Competition:
    """
    設定可能なTrueSkill競技環境。
    """
    
    def __init__(
        self,
        skill_dist: List[float],  # 各「良い」提出物に対するキーワードを見つける確率。
        nb_dummy: int,  # ダミーボットの数。
        mu: float = 600,  # TrueSkillのμパラメータ。
        sigma: float = 300,  # TrueSkillのσパラメータ。
        beta: float = 150,  # TrueSkillのβパラメータ。
        tau: float = 3,  # TrueSkillのτパラメータ。
        draw_probability: float = 0.9956,  # TrueSkillの引き分け確率パラメータ。
        seed: float = 42  # ランダムシード。
    ):
        """
        コンストラクタ。
        
        パラメータ
        ---------
            skill_dist: List[float]
                各「良い」提出物に対するキーワードを見つける確率。
            nb_dummy: int
                ダミーボットの数。
            mu: float
                TrueSkillのμパラメータ。
            sigma: float
                TrueSkillのσパラメータ。
            beta: float
                TrueSkillのβパラメータ。
            tau: float
                TrueSkillのτパラメータ。
            draw_probability: float
                TrueSkillの引き分け確率パラメータ。
            seed: float
                ランダムシード。
        """
        # 競技環境の設定
        self._trueskill = TrueSkill(
            mu = mu,
            sigma = sigma,
            beta = beta,
            tau = tau,
            draw_probability = draw_probability
        )
        
        # プレイヤープールの作成 — これがPlayerコンストラクタに「env」パラメータが必要だった理由です
        self.players = { ii: Player(s, self._trueskill) for ii,s in enumerate(skill_dist) }
        self.players.update({ len(self.players)+ii: Player(0, self._trueskill) for ii in range(nb_dummy) })  # ダミーボットを追加します。
        
        # ランダムシードの設定
        self._seed = seed
        random.seed(seed)
        
    def play_games(self, nb_games: int = 10000):
        """
        ゲームをシミュレートし、プレイヤーの履歴を更新します。
        
        パラメータ
        ---------
            nb_games: int
                プレイするゲームの数。
        """
        
        for i in tqdm(range(nb_games)):  # 指定された数のゲームをシミュレートします。
            player_ids = random.sample(range(len(self.players)), 4)  # ランダムに4人のプレイヤーを選びます。
            try:
                play_match(
                    [
                        self.players[player_ids[0]],  # チームAの最初のプレイヤー。
                        self.players[player_ids[1]]   # チームAの二番目のプレイヤー。
                    ],
                    [
                        self.players[player_ids[2]],  # チームBの最初のプレイヤー。
                        self.players[player_ids[3]]   # チームBの二番目のプレイヤー。
                    ],
                    env = self._trueskill  # 先ほど設定した環境を使用します。
                )
            except:
                continue  # エラーが発生した場合は次の反復に進みます。

以前に使用したスキル分布を回復しましょう。

In [ ]:
skill_dist = skills_df["skill"].values  # 以前に計算したスキル分布を取得します。
nb_dummy = len(players) - len(skill_dist)  # プレイヤーの総数から良いボットの数を引いてダミーボットの数を計算します。

In [ ]:
# LLM20の推測の設定に従いますが、比較するためにいくつかの競技を定義することもできます。
LLM20 = Competition(skill_dist, nb_dummy)  # LLM20コンペティションをインスタンス化します。

In [ ]:
# Kaggleのシステムでは、1秒あたり約1700ゲームを期待しています。
# 履歴がリセットされないことに注意してください。ゲームは以前のものに追加されます。
LLM20.play_games(100000)  # 100000ゲームをシミュレートします。

# プロット

このシミュレーションの結果に入る前に、実際の競技において、各提出物がプレイしたゲームの数の分布を念頭に置いておきましょう。これは競技開始から2ヶ月経過した後のものです：

In [ ]:
plt.figure(figsize=(12,6))  # プロットのサイズを設定します。
last_games_df.groupby("guesser_SubmissionId").size().hist(bins=15)  # 各提出物のゲーム数の分布をヒストグラムで表示します。
_ = plt.title("実際のプレイされたゲームの分布")  # プロットのタイトルを設定します。
_ = plt.xlabel("プレイされたゲームの数")  # x軸のラベルを設定します。
_ = plt.ylabel("ボットの数")  # y軸のラベルを設定します。

In [ ]:
last_games_df["guesser_SubmissionId"].nunique(), (last_games_df.groupby("guesser_SubmissionId").size()>100).sum()  # ユニークな「guesser_SubmissionId」の数と、100回を超えてプレイされた提出物の数を計算して表示します。

一部のボットは他のボットよりも多くのゲームをプレイしていますが、それは彼らがはるかに古いからです。実際、約半数のボットは100回未満のゲームをプレイしており、一部のボットは500回以上プレイしています！これは重要な点ですが、この後は無視することにしますが、実際の競技と比較する際には頭に入れておくと良いでしょう。

さて、シミュレーションのプロットを作成して、このランキングシステムが長期的にどのように機能するかを見てみましょう。

In [ ]:
fig = plt.figure(figsize=(16,8))  # プロットのサイズを設定します。
cmap = plt.get_cmap('viridis')  # カラーマップを取得します。

for i in range(len(LLM20.players))[::-1]:  # スキルが増加する順にプレイヤーをソートし、最良のものを上に置きます。
    pskill = LLM20.players[i].skill  # プレイヤーのスキルを取得します。
    plt.plot(
        [r.mu for r in LLM20.players[i].history],  # プレイヤーのレーティング履歴に対してプロットします。
        c = cmap(pskill) if pskill > 0 else [.6]*4  # スキルが0より大きい場合はカラーマップを使用し、ダミーボットは灰色で表示します。
    )
_ = plt.title("Muの進化")  # プロットのタイトルを設定します。
_ = plt.xlabel("プレイされたゲームの数")  # x軸のラベルを設定します。
_ = plt.ylabel("TrueSkillのμ値")  # y軸のラベルを設定します。

上のプロットは、*傾向がある*ものの、システムがプレイヤーのスキルに応じて区別することができないことを示しています。これは大量のゲームをプレイした後でもそうです。灰色の線はダミーボットに対応しています。

では、シミュレーションされた上位10人のプレイヤーの挙動を詳しく見てみましょう：

In [ ]:
NFIRSTS = 10  # 上位10人のプレイヤーを設定します。
fig = plt.figure(figsize=(16,8))  # プロットのサイズを設定します。
cmap = plt.get_cmap('tab10')  # タブカラーマップを取得します。

for i in range(NFIRSTS):
    pskill = LLM20.players[i].skill  # プレイヤーのスキルを取得します。
    plt.plot(
        [r.mu for r in LLM20.players[i].history],  # プレイヤーのレーティング履歴に対してプロットします。
        c = cmap(i),  # 各プレイヤーに異なる色を割り当てます。
        label = f"skill = {pskill:.2f}"  # レジェンド用のラベルを設定します。
    )
_ = plt.legend()  # レジェンドを表示します。
_ = plt.title("上位10人（シミュレートされた）プレイヤーのMuの進化")  # プロットのタイトルを設定します。
_ = plt.xlabel("プレイされたゲームの数")  # x軸のラベルを設定します。
_ = plt.ylabel("TrueSkillのμ値")  # y軸のラベルを設定します。

最良のプレイヤーたちは、スキルの低いプレイヤーと同等に競うのに苦労しています…

同じスキルを持つプレイヤーの進化を比較するのは興味深いかもしれません。特定の競技環境を設定してテストすることもできますが、私は最も現実的なシナリオを維持することに興味があるので、現在のプレイヤープールの中で確認してみましょう：

In [ ]:
from collections import Counter  # 集合のカウントを簡単に行うためのCounterをインポートします。

In [ ]:
duplicated_skills = Counter(skill_dist[:15]).most_common(2)  # スキル分布の上位15個をカウントし、最も一般的な2つを取得します。
duplicated_skills  # 重複したスキルのリストを表示します。

15人の最良プレイヤーの中に、興味深い候補が2組います。それらのパフォーマンスの推移を確認してみましょう。

In [ ]:
SHOW_ALL = True  # すべてのプレイヤーを表示するかどうかのフラグを設定します。
fig = plt.figure(figsize=(16,8))  # プロットのサイズを設定します。
cmap = plt.get_cmap('tab10')  # タブカラーマップを取得します。
gray = [.05] * 4  # ダミーボット用の灰色を設定します。

if SHOW_ALL:
    for i in range(len(LLM20.players)):
        plt.plot(
            [r.mu for r in LLM20.players[i].history],  # 各プレイヤーのレーティング履歴をプロットします。
            c = gray  # ダミーボットは灰色で表示します。
        )
for i in range(20):  # 上位20人のプレイヤーに対してループします。
    pskill = LLM20.players[i].skill  # プレイヤーのスキルを取得します。
    if pskill == duplicated_skills[0][0]:  # 最初の重複したスキルと一致する場合。
        color = cmap(0)  # カラーマップから色を取得します。
    elif pskill == duplicated_skills[1][0]:  # 二つ目の重複したスキルと一致する場合。
        color = cmap(1)  # カラーマップから別の色を取得します。
    else:
        continue  # 他はスキップします。
    
    plt.plot(
        [r.mu for r in LLM20.players[i].history],  # プレイヤーのレーティング履歴をプロットします。
        c = color,  # 適切な色で表示します。
        label = f"skill = {pskill:.2f}"  # レジェンド用のラベルを設定します。
    )
_ = plt.legend()  # レジェンドを表示します。
_ = plt.title("同一スキルを持つプレイヤーのMuの進化")  # プロットのタイトルを設定します。
_ = plt.xlabel("プレイされたゲームの数")  # x軸のラベルを設定します。
_ = plt.ylabel("TrueSkillのμ値")  # y軸のラベルを設定します。

スキルの高い競技者や低い競技者の間にはいくつかの変動がありますが、[@c-number](https://www.kaggle.com/competitions/llm-20-questions/discussion/520928)が報告したところでは、現在実際のリーダーボードで1位を獲得している彼は、3つの同一提出物が60の位置に分散していると言い、そのスコアは$\mu=1143$から…$\mu=767$まで変動していると述べています。この事実を考えると、そこまで悪くはありません！

# リーダーボードとの比較

シミュレーションされたリーダーボードが実際のものとどのように比較されるかを確認してみましょう：

In [ ]:
lb = pd.read_csv("/kaggle/input/llm-20-lb-2024-07-14/llm-20-questions-publicleaderboard-2024-07-14.csv")  # 実際のリーダーボードデータをCSVファイルから読み込みます。

In [ ]:
plt.figure(figsize=(12,6))  # プロットのサイズを設定します。
_ = plt.hist(
    lb.Score,
    bins = 100,  # 100のビンでヒストグラムを作成します。
    range = (300, 1200),  # スコアの範囲を300から1200に設定します。
    log = True,  # ログスケールで表示します。
    label = "リーダーボード"  # ヒストグラムのラベルを設定します。
)
_ = plt.hist(
    [ p.history[-1].mu for p in LLM20.players.values() ],  # シミュレーションされたプレイヤーの最終μ値を取得します。
    bins = 100,  # 100のビンでヒストグラムを作成します。
    range = (300, 1200),  # スコアの範囲を300から1200に設定します。
    alpha = .5,  # 透明度を設定します。
    log = True,  # ログスケールで表示します。
    label = "シミュレーション"  # ヒストグラムのラベルを設定します。
)
_ = plt.title("プレイされたゲームによる修正されたリーダーボード分布")  # プロットのタイトルを設定します。
_ = plt.xlabel("TrueSkill μ")  # x軸のラベルを設定します。
_ = plt.ylabel("ボットの数")  # y軸のラベルを設定します。
_ = plt.legend()  # レジェンドを表示します。

完璧とは言えませんが、そこまで悪くもありません。違いの一部はプレイされたゲームの数によって説明できますが、それだけでは十分ではありません。修正された比較を見てみましょう：

In [ ]:
plt.figure(figsize=(12,6))  # プロットのサイズを設定します。
played_dist = last_games_df.groupby("guesser_SubmissionId").size().values  # ゲームプレイ数を取得します。
_ = plt.hist(
    lb.Score,
    bins = 100,  # 100のビンでヒストグラムを作成します。
    range = (300, 1200),  # スコアの範囲を300から1200に設定します。
    log = True,  # ログスケールで表示します。
    label = "リーダーボード"  # ヒストグラムのラベルを設定します。
)
_ = plt.hist(
    [
        p.history[min(len(p.history)-1, random.choice(played_dist))].mu  # プレイされたゲームの分布から「最後のゲーム」の番号を選びます。
        for p in LLM20.players.values()
    ],
    bins = 100,  # 100のビンでヒストグラムを作成します。
    range = (300, 1200),  # スコアの範囲を300から1200に設定します。
    alpha = .5,  # 透明度を設定します。
    log = True,  # ログスケールで表示します。
    label = "シミュレーション"  # ヒストグラムのラベルを設定します。
)
_ = plt.title("プレイされたゲームによる修正されたリーダーボード分布")  # プロットのタイトルを設定します。
_ = plt.xlabel("TrueSkill μ")  # x軸のラベルを設定します。
_ = plt.ylabel("ボットの数")  # y軸のラベルを設定します。
_ =  plt.legend()  # レジェンドを表示します。

もう一つのポイントは、スキル分布が高い値にシフトしていることですが、再度、それだけでは差を説明するには不十分です。したがって、$\beta$や$\tau$の値がどこか間違っている可能性があると思います。また、競技中にパラメータが調整された可能性もあります。

いずれにせよ、私のテストから判断すると、このシミュレーションの結果が変わるとは強く疑っています。

# 上位10のゲーム履歴と比較

私は手動でリーダーボードの上位10位のゲーム履歴をスクレイピングし、データを辞書形式で処理しました。このデータは、[LLM-20-top10-LB-matches](https://www.kaggle.com/datasets/gguillard/llm-20-top10-lb-matches) データセット内のピクル形式で見つけることができます。

In [ ]:
import pickle  # ピクルをインポートします。オブジェクトのシリアライズに使用します。
with open("/kaggle/input/llm-20-top10-lb-matches/matches.pickle", "rb") as f:  # ピクルファイルを開きます。
    top10_games_dict = pickle.load(f)  # ピクルファイルからデータをロードします。

彼らの進化は私たちのシミュレーションとどのように比較されるでしょうか？

In [ ]:
plt.figure(figsize=(12,6))  # プロットのサイズを設定します。
gray = [.05] * 4  # ダミーボット用の灰色を設定します。
if SHOW_ALL:
    for i in range(len(LLM20.players)):
        plt.plot(
            [r.mu for r in LLM20.players[i].history[:200]],  # 各プレイヤーの最初の200ゲームのレーティング履歴をプロットします。
            c = gray  # ダミーボットは灰色で表示します。
        )
for kaggler, df in top10_games_dict.items():  # リーダーボードの上位10位のすべてのプレイヤーに対してループします。
    _ = plt.plot(df[kaggler], label = kaggler)  # 各プレイヤーのレーティング履歴をプロットします。
_ = plt.title("上位10のリーダーボードのμの進化（背景 = シミュレーション）")  # プロットのタイトルを設定します。
_ = plt.xlabel("プレイされたゲームの数")  # x軸のラベルを設定します。
_ = plt.ylabel("TrueSkillのμ値")  # y軸のラベルを設定します。
_ = plt.legend()  # レジェンドを表示します。

このプロットの目的は、リーダーボードの動きが私たちのシミュレーションと一致していることを示すことです。特に、SpiralTipの印象的な復活に注目してください…

# チームマッチング

このプロセスで考慮しなかった重要なポイントがあります。競技の概要より引用します：

> 各提出物は、リーダーボード上の同様のスキル評価を持つ他のボットと対戦するエピソード（ゲーム）をプレイします。

私の知る限り、TrueSkillライブラリにはプレイヤーをスキル評価に応じて引き出す便利な関数は提供されていません。このポイントを気にする価値はあるのでしょうか？ 上位10人のプレイヤーのゲーム履歴があるので、彼らが$\mu$の進化に沿ってどのようにペアリングされていたかを確認できます：

In [ ]:
plt.figure(figsize=(12,6))  # プロットのサイズを設定します。
for kaggler, df in top10_games_dict.items():  # 上位10人のプレイヤーに対してループします。
    _ = plt.scatter(df[kaggler], df["Op1"], s = 1, label = kaggler)  # プレイヤーのμとオポーネント1のμをプロットします。
    _ = plt.scatter(df[kaggler], df["Op2"], s = 1)  # オポーネント2のμをプロットします。
    _ = plt.scatter(df[kaggler], df["Op3"], s = 1)  # オポーネント3のμをプロットします。
_ = plt.legend()  # レジェンドを表示します。

いくつかの傾向は見られますが、そこからシミュレーション関数を推測するのは明らかではありません。さらに、高得点のボットがダミーボットとペアリングされる可能性があるため、彼らのレーティングは劣化する可能性があります（とはいえ、彼らの小さな $\sigma$ によってある程度は軽減されています）。

しかし、最も重要なことは、これは「[愚かさの穴](https://www.kaggle.com/competitions/llm-20-questions/discussion/514628#2889458)」の問題を解決するものではなく、むしろそれを助長することです。@loh-maaが非常に的確に名付けた問題です。

# 結論

このシミュレーションで恐らく不足している点が多くあります。特に、時間とともに新しい提出物があること、各プレイヤーのゲーム頻度が時間とともに減少すること（もしかしたら$\sigma$に関連しているかもしれません）、あるいはある程度までプレイヤーが同じランクのプレイヤーとマッチングされるという事実です。しかし、これらの「特徴」が私の結論を変えるとは思えません。現状では、LLM 20 Questionsコンペティションのランキングシステムは、パフォーマンスよりも偶然の要素を多く含んでいることは明らかです。

# プレイグラウンド

上記のすべてを(再)読み返すことなく、TrueSkill環境をシミュレーションしたいという方のために、必要な情報をすべてここにまとめました。

In [ ]:
!pip install trueskill  # TrueSkillライブラリをインストールします。これは、ボットのスキル評価を行うために必要です。

In [ ]:
import random  # ランダムな数値を生成するためのライブラリをインポートします。
from typing import List, Optional, Tuple  # 型注釈を提供するために必要な型をインポートします。
import warnings  # 警告を表示するためのライブラリをインポートします。

from tqdm import tqdm  # 進行状況バーを表示するためのライブラリをインポートします。
from trueskill import TrueSkill  # TrueSkillライブラリをインポートします。これは、レーティングシステムの一部です。

In [ ]:
class Player:
    def __init__(
        self,
        skill: float,  # プレイヤーのスキルを初期化します。
        env: Optional[TrueSkill] = None  # 今は無視しますが、後で役立ちます。
    ):
        self.skill = skill  # スキルを設定します。
        self.rating = Rating() if env is None else env.Rating()  # 環境のデフォルト値（600,300）で開始します。
        self.history = []  # レーティング履歴を保持するリストを初期化します。
        
    def update(self, new_rating):  # 新しいレーティングで更新するメソッドを定義します。
        self.history.append(self.rating)  # 現在のレーティングを履歴に追加します。
        self.rating = new_rating  # 新しいレーティングを設定します。

In [ ]:
def play_match(
    teamA: Tuple[Player, Player],  # チームAのプレイヤーを定義します。
    teamB: Tuple[Player, Player],  # チームBのプレイヤーを定義します。
    env: Optional[TrueSkill],  # 環境をオプションで設定します。
    debug: bool = False  # デバッグ用のフラグを設定します。
):
    """
    2つのチーム間でマッチをプレイします。
    
    パラメータ
    ---------
        teamA: Tuple[Player, Player]
            2人のプレイヤーからなるチームA。
        teamB: Tuple[Player, Player]
            2人のプレイヤーからなるチームB。
        debug: boolean
            デバッグ用のフラグ。
    """
    if debug:
        print("プレイヤーのスキル :", [p.skill for p in teamA+teamB])  # プレイヤーのスキルを表示します。
        print("プレイヤーのレーティング :", [tuple(p.rating for p in teamA), tuple(p.rating for p in teamB)])  # プレイヤーのレーティングを表示します。
    
    # 各チームのプレイヤーがダミーボットであれば、そのチームはキーワードを見つけられない
    # さもなくば、チームは推測者のスキルがランダムな数値よりも高ければキーワードを見つける
    # （コード内の逆のロジックに注意）
    ranks = [
        1 if teamA[0].skill * teamA[1].skill == 0 or random.random() > teamA[0].skill else 0,
        1 if teamB[0].skill * teamB[1].skill == 0 or random.random() > teamB[0].skill else 0
    ]
    
    # 新しいレーティングはゲームの結果です
    new_ratings = env.rate(
        [
            [ p.rating for p in teamA ],  # チームAのプレイヤーのレーティングを取得します。
            [ p.rating for p in teamB ]   # チームBのプレイヤーのレーティングを取得します。
        ],
        ranks  # 勝率を引数にします。
    )
    
    # 全てのプレイヤーのレーティングを更新します
    for p,r in zip(teamA, new_ratings[0]):
        p.update(r)  # チームAの各プレイヤーのレーティングを更新します。
    for p,r in zip(teamB, new_ratings[1]):
        p.update(r)  # チームBの各プレイヤーのレーティングを更新します。
        
    if debug:
        print("ゲーム結果 :", ranks)  # ゲームの結果を表示します。
        print("新しいレーティング :", new_ratings)  # 新しいレーティングを表示します。
    
    return  # 終了します。

In [ ]:
class Competition:
    """
    設定可能なTrueSkill競技環境。
    """
    
    def __init__(
        self,
        skill_dist: List[float],  # 各「良い」提出物に対するキーワードを見つける確率。
        nb_dummy: int,  # ダミーボットの数。
        mu: float = 600,  # TrueSkillのμパラメータ。
        sigma: float = 300,  # TrueSkillのσパラメータ。
        beta: float = 150,  # TrueSkillのβパラメータ。
        tau: float = 3,  # TrueSkillのτパラメータ。
        draw_probability: float = 0.9956,  # TrueSkillの引き分け確率パラメータ。
        seed: float = 42,  # ランダムシード。
        debug: bool = False  # デバッグフラグ。
    ):
        """
        コンストラクタ。
        
        パラメータ
        ---------
            skill_dist: List[float]
                各「良い」提出物に対するキーワードを見つける確率。
            nb_dummy: int
                ダミーボットの数。
            mu: float
                TrueSkillのμパラメータ。
            sigma: float
                TrueSkillのσパラメータ。
            beta: float
                TrueSkillのβパラメータ。
            tau: float
                TrueSkillのτパラメータ。
            draw_probability: float
                TrueSkillの引き分け確率パラメータ。
            seed: float
                ランダムシード。
            debug: bool
                デバッグ用のフラグ。
        """
        # 競技環境の設定
        self._trueskill = TrueSkill(
            mu = mu,
            sigma = sigma,
            beta = beta,
            tau = tau,
            draw_probability = draw_probability
        )
        
        # プレイヤープールの作成 — これがPlayerコンストラクタに「env」パラメータが必要だった理由です
        self.players = { ii: Player(s, self._trueskill) for ii,s in enumerate(skill_dist) }
        self.players.update({ len(self.players)+ii: Player(0, self._trueskill) for ii in range(nb_dummy) })  # ダミーボットを追加します。
        
        # ランダムシードの設定
        self._seed = seed
        random.seed(seed)
        
        self.debug = debug  # デバッグフラグを設定します。
        
    def play_games(self, nb_games: int = 10000):
        """
        ゲームをシミュレートし、プレイヤーの履歴を更新します。
        
        パラメータ
        ---------
            nb_games: int
                プレイするゲームの数。
        """
        for i in tqdm(range(nb_games)):  # 指定された数のゲームをシミュレートします。
            player_ids = random.sample(range(len(self.players)), 4)  # ランダムに4人のプレイヤーを選びます。
            try:
                play_match(
                    [
                        self.players[player_ids[0]],  # チームAの最初のプレイヤー。
                        self.players[player_ids[1]]   # チームAの二番目のプレイヤー。
                    ],
                    [
                        self.players[player_ids[2]],  # チームBの最初のプレイヤー。
                        self.players[player_ids[3]]   # チームBの二番目のプレイヤー。
                    ],
                    env = self._trueskill,  # 環境を使用します。
                    debug = self.debug  # デバッグフラグを渡します。
                )
            except Exception as e:  # エラーが発生した場合
                warnings.warn(f"Caught exception {e}")  # 警告を表示します。
                continue  # 次の反復に進みます。

In [ ]:
nb_players = 10  # 実際のプレイヤーの数
nb_dummy_players = 10  # ダミーボット（「悪い」ボット）の数
skill_distribution = [ random.random() for i in range(nb_players)]  # スキル分布をランダムに生成します。

In [ ]:
# いくつかの競技を定義して比較することができます。
C1 = Competition(skill_distribution, nb_dummy_players)  # デフォルト設定のコンペティションを作成します。
C2 = Competition(skill_distribution, nb_dummy_players, mu = 500, sigma = 100, beta = 200, tau = 10, draw_probability = .8, seed = 123)  # 異なるパラメータ設定のコンペティションを作成します。

In [ ]:
# Kaggleのシステムでは、1秒あたり約1700ゲームを期待しています。
# 履歴がリセットされないことに注意してください。ゲームは以前のものに追加されます。
C1.play_games(10000)  # C1のコンペティションで10000ゲームをシミュレートします。
C2.play_games(10000)  # C2のコンペティションで10000ゲームをシミュレートします。

In [ ]:
import matplotlib.pyplot as plt  # データの可視化のためにmatplotlib.pyplotをインポートします。これを使用してシミュレーションの結果をプロットします。

In [ ]:
plt.plot([ r.mu for r in C1.players[0].history])  # C1の最初のプレイヤーのレーティング履歴（μ値）をプロットします。

In [ ]:
# (No input provided. Please provide a new instruction or input to process.)

# シードによる変動性

In [ ]:
C = [Competition(skill_dist, nb_dummy, seed=i) for i in range(4)]  # 異なるシードを使用して4つの競技インスタンスを作成します。

In [ ]:
for comp in C:
    comp.play_games(50000)  # 各コンペティションインスタンスで50000ゲームをシミュレートします。

In [ ]:
def plot_first_n(comp, nfirsts, a):
    #fig = plt.figure(figsize=(16,8))  # プロットのサイズを設定します（コメントアウトされています）。
    cmap = plt.get_cmap('tab10')  # タブカラーマップを取得します。
    for i in range(nfirsts):
        pskill = comp.players[i].skill  # プレイヤーのスキルを取得します。
        a.plot(
            [r.mu for r in comp.players[i].history],  # プレイヤーのレーティング履歴をプロットします。
            c = cmap(i),  # 各プレイヤーに異なる色を割り当てます。
            label = f"skill = {pskill:.2f}"  # レジェンド用のラベルを設定します。
        )
    _ = a.legend()  # レジェンドを表示します。
    _ = a.set_title("上位10プレイヤーのMuの進化（シミュレーション）")  # プロットのタイトルを設定します。
    _ = a.set_xlabel("プレイされたゲームの数")  # x軸のラベルを設定します。
    _ = a.set_ylabel("TrueSkillのμ値")  # y軸のラベルを設定します。

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=1, figsize=(16,16), constrained_layout=True)  # 4つの行と1つの列を持つサブプロットを作成します。
for i, comp in enumerate(C):
    #plt.axes(ax[i])  # 現在の軸を設定します（コメントアウトされています）。
    plot_first_n(comp, 5, ax[i])  # 各コンペティションの上位5人のプレイヤーのMuの進化をプロットします。

In [ ]:
for comp in C:
    scores = {p: v.rating.mu for p,v in comp.players.items()}  # 各プレイヤーのμ値を取得します。
    leaderboard = {k:v for k,v in sorted(scores.items(), key=lambda item: item[1], reverse = True)}  # スコアに基づいてリーダーボードをソートします。
    print("Leaderboard")  # リーダーボードの見出しを印刷します。
    print("rank\tplayer\tskill\tmu")  # 列名を印刷します。
    for i in range(5):  # 上位5名のプレイヤーをループします。
        pnum = list(leaderboard.keys())[i]  # プレイヤーの番号を取得します。
        print(f"{i+1}\t{pnum}\t{comp.players[pnum].skill:.2f}\t{leaderboard[pnum]:.0f}")  # ランク、プレイヤー、スキル、μを印刷します。
    print("-------------")  # 区切りを印刷します。